In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import datetime

# Load dataset
df = pd.read_csv('dataset/new_dataset.csv')
df['valid_time'] = pd.to_datetime(df['valid_time'])
df.set_index('valid_time', inplace=True)

# Drop unwanted columns
df_clean = df.drop(columns=['latitude', 'longitude', 'number', 'expver'])

# Define columns for which normalization is needed
cols_to_normalize = [
    'u10', 'v10', 'sp',
    'tp', 'skt', 'ssrd', 
    'hcc', 'lcc', 'mcc', 'tcc', 
    'e', 'cp', 'lsp', 
    'ptype', 'sf', 'z'
]

# Normalize the selected columns and store scalers for inverse transformation
scalers = {}
df_normalized = df_clean.copy()

for col in cols_to_normalize:
    scaler = MinMaxScaler()
    df_normalized[col] = scaler.fit_transform(df_clean[[col]])
    scalers[col] = scaler

# Train-test split (80% train, 20% test)
train_size = int(len(df_normalized) * 0.8)
train, test = df_normalized[cols_to_normalize][:train_size], df_normalized[cols_to_normalize][train_size:]

# Fit ARIMA model for each target variable
def fit_arima_model(data, p, d, q):
    model = sm.tsa.ARIMA(data, order=(p, d, q))
    model_fit = model.fit()
    return model_fit

# Define ARIMA parameters (these can be optimized based on the data)
p, d, q = 5, 1, 0  # Example values

# Prepare storage for models and predictions
arima_models = {}
arima_predictions = {}

# Fit ARIMA for each normalized feature
for col in cols_to_normalize:
    arima_models[col] = fit_arima_model(train[col], p, d, q)

# Make predictions for the test set for each variable
for col, model in arima_models.items():
    # Generate predictions for the length of the test set
    start_index = len(train)
    end_index = start_index + len(test) - 1
    arima_predictions[col] = model.predict(start=start_index, end=end_index)

# Inverse transform the predictions for each variable separately
arima_predictions_inv = {}
for col in arima_predictions.keys():
    arima_predictions_inv[col] = scalers[col].inverse_transform(arima_predictions[col].values.reshape(-1, 1)).flatten()

# Calculate MSE for each variable
mse_results = {}
for col in arima_predictions.keys():
    actual_values = scalers[col].inverse_transform(test[col].values.reshape(-1, 1)).flatten()
    mse = mean_squared_error(actual_values, arima_predictions_inv[col])
    mse_results[col] = mse
    print(f"MSE for {col}: {mse:.2f}")




C:\Users\Charu Gupta\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Charu Gupta\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Charu Gupta\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Charu Gupta\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no

MSE for u10: 1.89
MSE for v10: 2.44
MSE for sp: 202578312.63
MSE for tp: 0.00
MSE for skt: 132.80
MSE for ssrd: 1240511907890.10
MSE for hcc: 0.12
MSE for lcc: 0.03
MSE for mcc: 0.04
MSE for tcc: 0.17
MSE for e: 0.00
MSE for cp: 0.00
MSE for lsp: 0.00
MSE for ptype: 1.25
MSE for sf: 0.00
MSE for z: 199044643.29
